# GAMgame to SON ontologies



In [1]:
import json , rdflib , hashlib , re , requests
import os.path
from langdetect import detect
from rdflib import *
from rdflib import URIRef, Literal, Namespace, Graph 
from rdflib.namespace import RDF , RDFS, DC , XSD
from rdflib.graph import Seq
from rdflib.serializer import Serializer
import pandas as pd
import numpy as np

def get_emotion(txt, filename, lang='it'):
    """CALL CELI emotion annotation APIs"""
    try:
        lang = detect(txt) if detect(txt) and detect(txt) == 'en' else 'it'
    except Exception as e:
        pass
    try:
        data = '{"content": "'+txt+'"}'
        annotations = requests.post('https://sophia-cluster-dev.aws.celi.it/'+lang+'/spice/analysis', auth=('live', 'analys1s0042'), data=data.encode('utf-8'))
        if annotations.status_code == 200:
            annotations_json = annotations.json()
            with open('emotions/'+filename+'.json', 'w') as f:
                json.dump(annotations_json, f)  
            print(txt, annotations_json) 
            return annotations_json
    except Exception as e:
        print(txt, e)
        pass


with open('rdf_transform/context.json') as context_arco:
    context_doc = json.load(context_arco)

    
    
    
g = Graph()
    
# namespaces
spice = "https://w3id.org/spice/"
base = spice + 'gam/'
ARCO = Namespace("https://w3id.org/arco/ontology/arco/")
ARCOCORE = Namespace("https://w3id.org/arco/ontology/core/")
ARCOCD = Namespace("https://w3id.org/arco/ontology/context-description/")
ARCODD = Namespace("https://w3id.org/arco/ontology/denotative-description/")
schema = Namespace("http://schema.org/")
earmark = Namespace("http://www.essepuntato.it/2008/12/earmark#")
EMOTION = Namespace("https://w3id.org/spice/SON/emotion/")
DUL = Namespace("http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#")
FC = Namespace("https://w3id.org/spice/SON/fruitionContext/")
MANIFEST = Namespace("https://w3id.org/spice/manifest/")
NARRATIVE = Namespace("https://w3id.org/spice/SON/Narrative-Labyrinth#")
SCRIPT = Namespace("https://w3id.org/spice/SON/scripting/")
semiotics = Namespace("http://ontologydesignpatterns.org/cp/owl/semiotics.owl#")
    
g.bind( "gam" , base)
g.bind( "dc" , DC)
g.bind( "arco" , ARCO)
g.bind( "arco-core" , ARCOCORE)
g.bind( "arco-cd" , ARCOCD)
g.bind( "arco-dd" , ARCODD)
g.bind( "schema" , schema)
g.bind( "emo" , EMOTION)
g.bind( "earmark" , earmark)
g.bind( "dul" , DUL)
g.bind( "fc" , FC)
g.bind( "nar" , NARRATIVE)
g.bind( "script" , SCRIPT)
g.bind( "semiotics" , semiotics)

# const
dateStart = "2020-11-25T00:00:00+00:00"
dateEnd = "2020-11-26T00:00:00+00:00"
script_code = "00003_script"
script_execution = spice+'gam_20202611/'+script_code
time_interval = spice+'gam_20202611/time_interval'

# GAM game data - live
selections = pd.read_csv("GAM_game/scelte.csv", engine='python', index_col=False, skipinitialspace=True, quotechar="'", sep=",")
sessions = pd.read_csv("GAM_game/sessioni.csv", engine='python', index_col=False, skipinitialspace=True, quotechar="'", sep=",")
recommendations = pd.read_csv("GAM_game/raccomandate.csv", engine='python', index_col=False, skipinitialspace=True, quotechar="'", sep=",")

# catalogue
with open("GAM_test_catalogue.json") as json_file:
    data = json.load(json_file)
    all_artefacts = [base+'artefact/'+artefact_dict["ID"] for artefact_dict in data ]
    
# SCRIPT EXECUTION
g.add(( URIRef(script_execution), RDF.type , SCRIPT.ScriptExecution  ))
g.add(( URIRef(script_execution), RDFS.label , Literal("GAM game at: La notte dei ricercatori, November 25-26, 2020. Turin.")  ))
g.add(( URIRef(script_execution), SCRIPT.occursWithin , URIRef(spice+'event/notte_dei_ricercatori_2020_turin')  ))
g.add(( URIRef(script_execution), SCRIPT.satisfies , URIRef(MANIFEST+"00003_script")  ))

g.add(( URIRef(time_interval), RDF.type, DUL.TimeInterval ))
g.add(( URIRef(time_interval), DUL.hasIntervalDate, Literal( dateStart, datatype=XSD.dateTime) ))
g.add(( URIRef(time_interval), DUL.hasIntervalDate, Literal( dateEnd, datatype=XSD.dateTime) ))

sessions_list = []
# preparation: the ordered list of artefacts presented in this session
for index, session in sessions.iterrows():
    code = session["codice"]
    sessions_list.append(code)

    # the ordered list of artefacts shown in the application
    artefact_list =  [base+'artefact/'+ses.replace(' ','').strip() for ses in str(session["ordinamento_iniziale"]).split(', ') if len(str(session["ordinamento_iniziale"])) and len(ses.strip())>0]
    preparation_action = script_execution+'/preparation/'+code
    curatorial_selection = script_execution+'/preparation/'+code+'/curatorial_selection'
    software_presentation = script_execution+'/preparation/'+code+'/app_presentation'
    presentation_action = script_execution+'/presentation/'+code

    # 1. for each session we have an action preparation stage > selection of artefacts
    g.add(( URIRef(script_execution), SCRIPT.includesAction , URIRef(preparation_action)  ))
    g.add(( URIRef(preparation_action) , RDF.type , SCRIPT.Action  ))
    g.add(( URIRef(preparation_action) , SCRIPT.executesTask , URIRef(MANIFEST+'00003_artefact_selection')  ))  
    g.add(( URIRef(preparation_action) , SCRIPT.hasTimeInterval , URIRef(time_interval)  ))

    g.add(( URIRef(preparation_action) , SCRIPT.hasRoleInTime , URIRef(curatorial_selection)  )) 
    g.add(( URIRef(curatorial_selection) , RDF.type , SCRIPT.RoleInTime  )) 
    g.add(( URIRef(curatorial_selection) , SCRIPT.isRoleInTimeOf , URIRef(script_execution+'/curator')  )) 
    g.add(( URIRef(curatorial_selection) , SCRIPT.withRole , SCRIPT.curator  )) 
    g.add(( URIRef(curatorial_selection) , SCRIPT.hasTimeInterval , URIRef(time_interval)  ))  


    for art in all_artefacts: # use the unordered list of artefacts
        g.add(( URIRef(preparation_action) , SCRIPT.used , URIRef(art) )) 
        g.add(( URIRef(art) , RDF.type , ARCO.CulturalProperty ))
        g.add(( URIRef(art) , SCRIPT.correspondsTo , URIRef(MANIFEST+'00003_gam_artefact') ))

    # generate an ordered list of images - as defined in artefact_list    
    init_imgs = [URIRef(o_art+"/img_1") for o_art in artefact_list]
    #seq_art = preparation_action+'/initial_set'
    #initial_set = Seq(graph=g, uri=seq_art, seq=init_imgs)
    #g.add(( URIRef(preparation_action) , SCRIPT.generated , URIRef(seq_art) ))
    for im in init_imgs:
        g.add(( URIRef(preparation_action) , SCRIPT.generated , im ))  
        g.add(( im , SCRIPT.correspondsTo , URIRef(MANIFEST+'00003_gam_image') ))  

    g.add(( URIRef(preparation_action) , SCRIPT.precedes , URIRef(presentation_action)  ))

    # 2. for each session we have an action running stage > presentation of pictures
    g.add(( URIRef(script_execution), SCRIPT.includesAction , URIRef(presentation_action)  )) 
    g.add(( URIRef(presentation_action) , RDF.type , SCRIPT.Action  ))
    g.add(( URIRef(presentation_action) , SCRIPT.executesTask , URIRef(MANIFEST+'00003_presentation')  )) 
    g.add(( URIRef(presentation_action) , SCRIPT.hasTimeInterval , URIRef(time_interval)  ))
    g.add(( URIRef(presentation_action) , SCRIPT.hasRoleInTime , URIRef(software_presentation)  )) 
    g.add(( URIRef(software_presentation) , RDF.type , SCRIPT.RoleInTime  )) 
    g.add(( URIRef(software_presentation) , SCRIPT.isRoleInTimeOf , URIRef(spice+'gamgame_webapp')  )) 
    g.add(( URIRef(software_presentation) , SCRIPT.withRole , URIRef(SCRIPT+'software-agent') )) 
    g.add(( URIRef(software_presentation) , SCRIPT.hasTimeInterval , URIRef(time_interval)  ))
    for im in init_imgs: # use the ordered list of pictures
        g.add(( URIRef(software_presentation) , SCRIPT.used , im ))  
    g.add(( URIRef(presentation_action) , SCRIPT.follows , URIRef(preparation_action)  ))


# 3. for each session, multiple selection of images during a session
for code in sessions_list:
    picks = selections.loc[selections['codice_sessione'] == code ]
    picks = picks.replace(['NaN', 'None'], '').replace(np.nan, '', regex=True)
    for j, (index, pick) in enumerate(picks.iterrows()):        
        i = j+1 
        art_id= str(pick["id_opera"])  
        pick_action = script_execution+'/pick_'+str(i)+'/'+code
        artefact_uri = base+'artefact/'+art_id  
        selected_image = [artefact_uri+'/img_1' for art in data if art["ID"] == art_id ][0]
        pick_text_clean = str(pick["testo"]).replace("Mi fa sentire:","").replace("Mi ricorda:","").replace("Mi fa pensare a:","")
        text_story = script_execution+'/write_text_'+str(i)+'/'+code 
        text_emoji = script_execution+'/write_emo_'+str(i)+'/'+code 
        text_tag = script_execution+'/write_tag_'+str(i)+'/'+code 
        
        # 3.1 pick an image
        user_selection = pick_action+'/user_selection'  
        g.add(( URIRef(script_execution), SCRIPT.includesAction , URIRef(pick_action)  )) 
        g.add(( URIRef(pick_action) , RDF.type , SCRIPT.Action  ))
        g.add(( URIRef(pick_action) , SCRIPT.executesTask , URIRef(MANIFEST+'00003_user_picture_selection')  )) 
        g.add(( URIRef(pick_action) , SCRIPT.hasTimeInterval , URIRef(time_interval)  ))
        g.add(( URIRef(pick_action) , SCRIPT.hasRoleInTime , URIRef(user_selection)  )) 
        g.add(( URIRef(user_selection) , RDF.type , SCRIPT.RoleInTime  )) 
        g.add(( URIRef(user_selection) , SCRIPT.isRoleInTimeOf , URIRef(spice+'user/'+code )  )) 
        g.add(( URIRef(user_selection) , SCRIPT.withRole , URIRef(SCRIPT+'web-user') )) 
        g.add(( URIRef(user_selection) , SCRIPT.hasTimeInterval , URIRef(time_interval)  ))
        for im in init_imgs: # use the ordered list of pictures
            g.add(( URIRef(pick_action) , SCRIPT.used , im ))
        g.add(( URIRef(pick_action) , SCRIPT.generated , URIRef(selected_image) ))
        g.add(( URIRef(selected_image) , SCRIPT.correspondsTo , URIRef(MANIFEST+'00003_gam_selected_image')  ))
        
        if j < len(picks) - 1 and text_story is None and text_tag is None and text_emoji is None:
                next_pick_action = script_execution+'/pick_'+str(i+1)+'/'+code
                g.add(( URIRef(pick_action) , SCRIPT.precedes , URIRef(next_pick_action)  ))
                g.add(( URIRef(next_pick_action) , SCRIPT.follows , URIRef(pick_action)  ))

        # 3.2 if the user writes a text   
        user_answering = pick_action+'/user_answering'
        user_story = pick_action+'/user_story'
        g.add(( URIRef(script_execution), SCRIPT.includesAction , URIRef(text_story)  )) 
        g.add(( URIRef(pick_action) , SCRIPT.precedes , URIRef(text_story)  ))
        g.add(( URIRef(text_story) , SCRIPT.follows , URIRef(pick_action)  ))

        g.add(( URIRef(text_story) , RDF.type , SCRIPT.Action  ))
        g.add(( URIRef(text_story) , SCRIPT.executesTask , URIRef(MANIFEST+'00003_text_answering')  )) 
        g.add(( URIRef(text_story) , SCRIPT.hasTimeInterval , URIRef(time_interval)  ))
        g.add(( URIRef(text_story) , SCRIPT.hasRoleInTime , URIRef(user_answering)  )) 
        g.add(( URIRef(user_answering) , RDF.type , SCRIPT.RoleInTime  )) 
        g.add(( URIRef(user_answering) , SCRIPT.isRoleInTimeOf , URIRef(spice+'user/'+code )  )) 
        g.add(( URIRef(user_answering) , SCRIPT.withRole , URIRef(SCRIPT+'web-user') ))
        g.add(( URIRef(user_answering) , SCRIPT.withRole , URIRef(SCRIPT+'story-teller') ))
        g.add(( URIRef(user_answering) , SCRIPT.hasTimeInterval , URIRef(time_interval)  ))
        g.add(( URIRef(text_story) , SCRIPT.used , URIRef(selected_image) ))
        if len(str(pick_text_clean)) > 0:
            g.add(( URIRef(text_story) , SCRIPT.generated , URIRef(user_story) ))
            g.add(( URIRef(user_story) , RDF.type , earmark.StringDocuverse  ))
            g.add(( URIRef(user_story) , SCRIPT.correspondsTo , URIRef(MANIFEST+'00003_user_text')  ))
            g.add(( URIRef(user_story) , earmark.hasContent , Literal( str(pick["testo"] ) )  ))
            
            # CALL CELI emotion annotation APIs
            if os.path.isfile('emotions/gam_story_'+art_id+'_'+code+'.json'):
                print("it exists: ",'emotions/gam_story_'+art_id+'_'+code+'.json')
                with open('emotions/gam_story_'+art_id+'_'+code+'.json') as f:
                    annotations_json = json.load(f)
            #else:
            #    txt_st = str(pick["testo"]).replace("#",'').replace("\n",' ').replace("\r",' ').replace('"', '-')
            #    annotations_json = get_emotion(txt_st , 'gam_story_'+art_id+'_'+code) 

            if annotations_json and len(annotations_json["@graph"])> 1:
                for ann in annotations_json["@graph"]:
                    l = 0
                    if "@type" in ann \
                        and ann["@type"] == "earmark:PointerRange" \
                        and "semiotics:denotes" in ann \
                        and "@type" in ann["semiotics:denotes"] \
                        and "emotion:" in ann["semiotics:denotes"]["@type"]:
                        cur_emotion = ann["semiotics:denotes"]["@id"][3:]
                        cur_emotion_type = ann["semiotics:denotes"]["@type"].split(":")[1]
                        l += 1
                        g.add(( URIRef(user_story+'/pointer_range_'+str(l)), earmark.refersTo , URIRef(user_story) ))
                        g.add(( URIRef(user_story+'/pointer_range_'+str(l)), semiotics.denotes , URIRef(user_story+'/pointer_range_'+str(l)+'/'+cur_emotion) ))
                        g.add(( URIRef(user_story+'/pointer_range_'+str(l)+'/'+cur_emotion), RDF.type , URIRef(EMOTION+cur_emotion_type) ))
                        g.add(( URIRef(artefact_uri+'/'+code+'/emotion_relation'), EMOTION.emotion, URIRef(user_story+'/pointer_range_'+str(l)+'/'+cur_emotion) ))
                        g.add(( URIRef(artefact_uri), EMOTION.triggers, URIRef(user_story+'/pointer_range_'+str(l)+'/'+cur_emotion) ))             


        if j < len(picks) - 1 and text_tag is None and text_emoji is None:
            next_pick_action = script_execution+'/pick_'+str(i+1)+'/'+code
            g.add(( URIRef(text_story) , SCRIPT.precedes , URIRef(next_pick_action)  ))
            g.add(( URIRef(next_pick_action) , SCRIPT.follows , URIRef(text_story)  ))
                
        # 3.3 add an emoticon
        user_expressing = pick_action+'/user_expressing_emotion'
        user_emotion = pick_action+'/user_emotion'
        g.add(( URIRef(script_execution), SCRIPT.includesAction , URIRef(text_emoji)  )) 
        if text_story:
            g.add(( URIRef(text_story) , SCRIPT.precedes , URIRef(text_emoji)  ))
            g.add(( URIRef(text_emoji) , SCRIPT.follows , URIRef(text_story)  ))
        else:
            g.add(( URIRef(pick_action) , SCRIPT.precedes , URIRef(text_emoji)  ))
            g.add(( URIRef(text_emoji) , SCRIPT.follows , URIRef(pick_action)  ))

        g.add(( URIRef(text_emoji) , RDF.type , SCRIPT.Action  ))
        g.add(( URIRef(text_emoji) , SCRIPT.executesTask , URIRef(MANIFEST+'00003_emoji_answering')  )) 
        g.add(( URIRef(text_emoji) , SCRIPT.hasTimeInterval , URIRef(time_interval)  ))
        g.add(( URIRef(text_emoji) , SCRIPT.hasRoleInTime , URIRef(user_expressing)  )) 
        g.add(( URIRef(user_expressing) , RDF.type , SCRIPT.RoleInTime  )) 
        g.add(( URIRef(user_expressing) , SCRIPT.isRoleInTimeOf , URIRef(spice+'user/'+code )  )) 
        g.add(( URIRef(user_expressing) , SCRIPT.withRole , URIRef(SCRIPT+'web-user') )) 
        g.add(( URIRef(user_expressing) , SCRIPT.withRole , URIRef(SCRIPT+'emotion-sharer') )) 
        g.add(( URIRef(user_expressing) , SCRIPT.hasTimeInterval , URIRef(time_interval)  ))
        g.add(( URIRef(text_emoji) , SCRIPT.used , URIRef(selected_image) ))
        if len(str(pick["emoticon"])) > 0:
            g.add(( URIRef(text_emoji) , SCRIPT.generated , URIRef(user_emotion) ))
            g.add(( URIRef(user_emotion) , RDF.type , earmark.StringDocuverse  ))
            g.add(( URIRef(user_emotion) , earmark.hasContent , Literal(str(pick["emoticon"]) )  ))
            g.add(( URIRef(user_emotion) , SCRIPT.correspondsTo , URIRef(MANIFEST+'00003_user_emoji')  ))
            
            # CALL CELI emotion annotation APIs
            if os.path.isfile('emotions/gam_emoji_'+art_id+'_'+code+'.json'):
                print("it exists: ",'emotions/gam_emoji_'+art_id+'_'+code+'.json')
                with open('emotions/gam_emoji_'+art_id+'_'+code+'.json') as f:
                    annotations_json = json.load(f)
            #else:
            #    txt_emo = str(pick["emoticon"]).replace("#",'').replace("\n",' ').replace("\r",' ').replace('"', '-')
            #    annotations_json = get_emotion(txt_emo, 'gam_emoji_'+art_id+'_'+code) 

            if annotations_json and len(annotations_json["@graph"])> 1:
                for ann in annotations_json["@graph"]:
                    l = 0
                    if "@type" in ann \
                        and ann["@type"] == "earmark:PointerRange" \
                        and "semiotics:denotes" in ann \
                        and "@type" in ann["semiotics:denotes"] \
                        and "emotion:" in ann["semiotics:denotes"]["@type"]:
                        cur_emotion = ann["semiotics:denotes"]["@id"][3:]
                        cur_emotion_type = ann["semiotics:denotes"]["@type"].split(":")[1]
                        l += 1
                        g.add(( URIRef(user_emotion+'/pointer_range_'+str(l)), earmark.refersTo , URIRef(user_emotion) ))
                        g.add(( URIRef(user_emotion+'/pointer_range_'+str(l)), semiotics.denotes , URIRef(user_emotion+'/pointer_range_'+str(l)+'/'+cur_emotion) ))
                        g.add(( URIRef(user_emotion+'/pointer_range_'+str(l)+'/'+cur_emotion), RDF.type , URIRef(EMOTION+cur_emotion_type) ))
                        g.add(( URIRef(artefact_uri+'/'+code+'/emotion_relation'), EMOTION.emotion, URIRef(user_emotion+'/pointer_range_'+str(l)+'/'+cur_emotion) ))
                        g.add(( URIRef(artefact_uri), EMOTION.triggers, URIRef(user_emotion+'/pointer_range_'+str(l)+'/'+cur_emotion) ))             


        if j < len(picks) - 1 and text_tag is None:
            next_pick_action = script_execution+'/pick_'+str(i+1)+'/'+code
            g.add(( URIRef(text_emoji) , SCRIPT.precedes , URIRef(next_pick_action)  ))
            g.add(( URIRef(next_pick_action) , SCRIPT.follows , URIRef(text_emoji)  ))
           
        # 3.4 add a hashtag  
        user_tagging = pick_action+'/user_tagging'
        user_tag = pick_action+'/user_tag'
        g.add(( URIRef(script_execution), SCRIPT.includesAction , URIRef(text_tag)  )) 
        if text_emoji:
            g.add(( URIRef(text_emoji) , SCRIPT.precedes , URIRef(text_tag)  ))
            g.add(( URIRef(text_tag) , SCRIPT.follows , URIRef(text_emoji)  ))
        elif text_emoji is None and text_story:
            g.add(( URIRef(text_story) , SCRIPT.precedes , URIRef(text_tag)  ))
            g.add(( URIRef(text_tag) , SCRIPT.follows , URIRef(text_story)  ))
        else:
            g.add(( URIRef(pick_action) , SCRIPT.precedes , URIRef(text_tag)  ))
            g.add(( URIRef(text_tag) , SCRIPT.follows , URIRef(pick_action)  ))

        if j < len(picks) - 1:
            next_pick_action = script_execution+'/pick_'+str(i+1)+'/'+code
            g.add(( URIRef(text_tag) , SCRIPT.precedes , URIRef(next_pick_action)  ))
            g.add(( URIRef(next_pick_action) , SCRIPT.follows , URIRef(text_tag)  ))

        g.add(( URIRef(text_tag) , RDF.type , SCRIPT.Action  ))
        g.add(( URIRef(text_tag) , SCRIPT.executesTask , URIRef(MANIFEST+'00003_hashtag_answering')  )) 
        g.add(( URIRef(text_tag) , SCRIPT.hasTimeInterval , URIRef(time_interval)  ))
        g.add(( URIRef(text_tag) , SCRIPT.hasRoleInTime , URIRef(user_tagging)  )) 
        g.add(( URIRef(user_tagging) , RDF.type , SCRIPT.RoleInTime  )) 
        g.add(( URIRef(user_tagging) , SCRIPT.isRoleInTimeOf , URIRef(spice+'user/'+code )  )) 
        g.add(( URIRef(user_tagging) , SCRIPT.withRole , URIRef(SCRIPT+'web-user') )) 
        g.add(( URIRef(user_tagging) , SCRIPT.withRole , URIRef(SCRIPT+'tagger') )) 
        g.add(( URIRef(user_tagging) , SCRIPT.hasTimeInterval , URIRef(time_interval)  ))
        g.add(( URIRef(text_tag) , SCRIPT.used , URIRef(selected_image) ))
        if len(str(pick["hashtag"])) > 0:
            g.add(( URIRef(text_tag) , SCRIPT.generated , URIRef(user_tag) ))
            g.add(( URIRef(user_tag) , RDF.type , earmark.StringDocuverse  ))
            g.add(( URIRef(user_tag) , earmark.hasContent , Literal(str(pick["hashtag"]) )  ))
            g.add(( URIRef(user_tag) , SCRIPT.correspondsTo , URIRef(MANIFEST+'00003_user_hashtag')  ))
            
            # CALL CELI emotion annotation APIs
            if os.path.isfile('emotions/gam_hashtag_'+art_id+'_'+code+'.json'):
                print("it exists: ",'emotions/gam_hashtag_'+art_id+'_'+code+'.json')
                with open('emotions/gam_hashtag_'+art_id+'_'+code+'.json') as f:
                    annotations_json = json.load(f)
            #else:
            #    txt_hashtag = str(pick["hashtag"]).replace("#",'').replace("\n",' ').replace("\r",' ').replace('"', '-')
            #    annotations_json = get_emotion(txt_hashtag , 'gam_hashtag_'+art_id+'_'+code) 

            if annotations_json and len(annotations_json["@graph"])> 1:
                for ann in annotations_json["@graph"]:
                    l = 0
                    if "@type" in ann \
                        and ann["@type"] == "earmark:PointerRange" \
                        and "semiotics:denotes" in ann \
                        and "@type" in ann["semiotics:denotes"] \
                        and "emotion:" in ann["semiotics:denotes"]["@type"]:
                        cur_emotion = ann["semiotics:denotes"]["@id"][3:]
                        cur_emotion_type = ann["semiotics:denotes"]["@type"].split(":")[1]
                        l += 1
                        g.add(( URIRef(user_tag+'/pointer_range_'+str(l)), earmark.refersTo , URIRef(user_tag) ))
                        g.add(( URIRef(user_tag+'/pointer_range_'+str(l)), semiotics.denotes , URIRef(user_tag+'/pointer_range_'+str(l)+'/'+cur_emotion) ))
                        g.add(( URIRef(user_tag+'/pointer_range_'+str(l)+'/'+cur_emotion), RDF.type , URIRef(EMOTION+cur_emotion_type) ))
                        g.add(( URIRef(artefact_uri+'/'+code+'/emotion_relation'), EMOTION.emotion, URIRef(user_tag+'/pointer_range_'+str(l)+'/'+cur_emotion) ))
                        g.add(( URIRef(artefact_uri), EMOTION.triggers, URIRef(user_tag+'/pointer_range_'+str(l)+'/'+cur_emotion) ))             


            
            
        # sequence of actions
        
        # the first round
        if j == 0:
            presentation_action = script_execution+'/presentation/'+code
            g.add(( URIRef(presentation_action) , SCRIPT.precedes , URIRef(pick_action)  ))     
        
        if j > 0 and j < len(picks) - 1:
            next_pick_action = script_execution+'/pick_'+str(i+1)+'/'+code
            g.add(( URIRef(text_tag) , SCRIPT.precedes , URIRef(next_pick_action)  ))
            
        # the last round, get the recommendations (if the user got there)
        if j == len(picks) - 1:
            all_recommendations = recommendations.loc[(recommendations['codice_sessione'] == code) & (recommendations['accettata'].isin([0,1]) )]
            for j, (index, recomm) in enumerate(all_recommendations.iterrows()):  
                recommend_action = script_execution+'/recommendation_'+str(j+1)+'/'+code
                gam_recommendation = script_execution+'/recommendation_'+str(j+1)+'/'+code+'/gam_recommendation'
                recommended_art = base+'artefact/'+str(recomm["id_opera"])+'/img_1'
                    
                g.add(( URIRef(script_execution), SCRIPT.includesAction , URIRef(recommend_action)  )) 
                g.add(( URIRef(text_tag) , SCRIPT.precedes , URIRef(recommend_action) ))
                g.add(( URIRef(recommend_action) , SCRIPT.follows , URIRef(text_tag) ))
                
                g.add(( URIRef(recommend_action) , RDF.type , SCRIPT.Action  ))
                g.add(( URIRef(recommend_action) , SCRIPT.executesTask , URIRef(MANIFEST+'00003_picture_recommendation')  )) 
                g.add(( URIRef(recommend_action) , SCRIPT.hasTimeInterval , URIRef(time_interval)  ))
                g.add(( URIRef(recommend_action) , SCRIPT.hasRoleInTime , URIRef(gam_recommendation)  )) 
                
                g.add(( URIRef(gam_recommendation) , RDF.type , SCRIPT.RoleInTime  )) 
                g.add(( URIRef(gam_recommendation) , SCRIPT.isRoleInTimeOf , URIRef(spice+'user/'+code )  ))
                g.add(( URIRef(gam_recommendation) , SCRIPT.isRoleInTimeOf , URIRef(spice+'gamgame_webapp' )  ))
                g.add(( URIRef(gam_recommendation) , SCRIPT.withRole , URIRef(SCRIPT+'recommender') ))
                g.add(( URIRef(gam_recommendation) , SCRIPT.hasTimeInterval , URIRef(time_interval)  ))
                
                g.add(( URIRef(recommend_action) , SCRIPT.used , URIRef(recommended_art) ))
                g.add(( URIRef(recommended_art) , SCRIPT.correspondsTo , URIRef(MANIFEST+'00003_gam_recommended_image')  ))
                
                user_answer = recommend_action+"/accepted" if recomm['accettata'] == 1 else recommend_action+"/rejected"
                g.add(( URIRef(recommend_action) , SCRIPT.generated , URIRef(user_answer) ))
                g.add(( URIRef(user_answer) , RDF.type , earmark.StringDocuverse  ))
                g.add(( URIRef(user_answer) , SCRIPT.correspondsTo , URIRef(MANIFEST+'00003_user_acceptance')  ))
                
                
                
with open("GAM_test_catalogue.json") as json_file:
    data = json.load(json_file)
    # FRUITION CONTEXT 
    for artefact_dict in data:        
        art_id = artefact_dict["ID"]
        artefact = base+'artefact/'+art_id
        
        # in every session only a few selected artefacts are annotated
        sessions_codes = selections.loc[selections['id_opera'] == int(art_id)]['codice_sessione']
        for code in sessions_codes: 
            g.add(( URIRef(spice+'gam_session/'+code), RDF.type , FC.RemoteFruitionContext  ))
            g.add(( URIRef(spice+'gam_session/'+code), FC.fruitedEntity , URIRef(artefact)  ))
            g.add(( URIRef(spice+'gam_session/'+code), DUL.hasParticipant , URIRef(spice+'user/'+code)  ))
            g.add(( URIRef(spice+'gam_session/'+code), FC.hasModality , FC.Visual  ))
            g.add(( URIRef(spice+'gam_session/'+code), FC.isSupportedBy , URIRef(spice+'gamgame_webapp')  ))
            g.add(( URIRef(spice+'gamgame_webapp'), RDF.type , FC.Device ))
            g.add(( URIRef(spice+'gam_session/'+code), DUL.hasTimeInterval , URIRef(time_interval)  ))
            
            # event: TODO RDF.type event?
            g.add(( URIRef(spice+'gam_session/'+code), FC.occursWithin , URIRef(spice+'event/notte_dei_ricercatori_2020_turin')  ))
            
            # Emotion relation and annotations
            g.add(( URIRef(artefact+'/'+code+'/emotion_relation'), FC.hasFruitionContext , URIRef(spice+'gam_session/'+code)  ))
            # TODO add CELI emotion Relation and annotations
            
            
g.serialize(destination='rdf_transform/GAMgame.json', format='json-ld', context=context_doc, encoding='utf-8')

it exists:  emotions/gam_hashtag_29_0065FA674B05E773F456D6B5E4EB4F19.json
it exists:  emotions/gam_story_41_0065FA674B05E773F456D6B5E4EB4F19.json
it exists:  emotions/gam_hashtag_41_0065FA674B05E773F456D6B5E4EB4F19.json
it exists:  emotions/gam_hashtag_47_0065FA674B05E773F456D6B5E4EB4F19.json
it exists:  emotions/gam_story_34_052DB8F168C72F5C9585BB4FC9107E42.json
it exists:  emotions/gam_story_42_052DB8F168C72F5C9585BB4FC9107E42.json
it exists:  emotions/gam_story_69_052DB8F168C72F5C9585BB4FC9107E42.json
it exists:  emotions/gam_story_45_067B5739C70320F3A8C7A42017F74828.json
it exists:  emotions/gam_hashtag_42_07151E2EE764F49646D9248FF5AEBC39.json
it exists:  emotions/gam_hashtag_25_0E8CC602A1065EFCFF5F5861EC9672B7.json
it exists:  emotions/gam_story_26_0F62B4DDBE8AACB7AF02F04CB266AE09.json
it exists:  emotions/gam_story_60_0F62B4DDBE8AACB7AF02F04CB266AE09.json
it exists:  emotions/gam_story_37_103B1AAC7F793F517A8428E9B05ECBA5.json
it exists:  emotions/gam_hashtag_41_10F1ECD350939C2F15

it exists:  emotions/gam_hashtag_34_84B50C8A1B43D05DE8A53E8624E5E50F.json
it exists:  emotions/gam_story_59_84B50C8A1B43D05DE8A53E8624E5E50F.json
it exists:  emotions/gam_emoji_25_85BF14887D78029252A6EC4052C82FAB.json
it exists:  emotions/gam_hashtag_25_85BF14887D78029252A6EC4052C82FAB.json
it exists:  emotions/gam_emoji_37_85BF14887D78029252A6EC4052C82FAB.json
it exists:  emotions/gam_hashtag_37_85BF14887D78029252A6EC4052C82FAB.json
it exists:  emotions/gam_story_60_863383DE7CF68B15E27105FC12BF95A5.json
it exists:  emotions/gam_hashtag_60_863383DE7CF68B15E27105FC12BF95A5.json
it exists:  emotions/gam_emoji_40_8799AC8F0B11E27E3BF2691ACFF69F42.json
it exists:  emotions/gam_story_53_89AEFA7F02F5CC458E48B87479B3D544.json
it exists:  emotions/gam_hashtag_62_89AEFA7F02F5CC458E48B87479B3D544.json
it exists:  emotions/gam_story_47_8A56096649E3552CE8A42E0DBC508F46.json
it exists:  emotions/gam_story_39_8FCB7FE1F13A942684BD2356C5ABD82C.json
it exists:  emotions/gam_story_41_8FCB7FE1F13A942684BD